In [1]:
from collections import defaultdict
from pathlib import Path
import polars as pl
from hierarchical import df_to_records, Record, path_to_root
from images_sorting import sort_images
from tests import test_sort_images
import os

In [2]:
path = Path(r"/mnt/z/DATASETS/ukraine/russia/russia/")
df = pl.read_csv(r"/mnt/z/DATASETS/ukraine/losses_russia.csv")
df_2 = df.groupby("model").agg(pl.col("losses_total").sum())

In [3]:
img_index = defaultdict(list)
for cat, sub_index in sort_images(df, path).items():
    img_index |= sub_index
rec = df_to_records(df_2, img_index)

In [5]:
def count_rec(r: Record):
    count = r.losses_total if r.losses_total else 0
    for child in r.children:
        count += count_rec(child)
    return count
count_rec(rec)

9022

In [6]:
def find_by_name(r: Record, n:str):
    if r.name==n:
        return r
    for child in r.children:
        if f:=find_by_name(child, n):
            return f
    return None

In [9]:
with open("../frontend/static/losses_russia.json", "w") as fp:
    fp.write(rec.json())